In [3]:
import pandas as pd
import hashlib

In [10]:
def find_unique_texts(source_file, comparison_file, output_file, num_samples=500):
    """
    Находит тексты из source_file, которых нет в comparison_file, и сохраняет первые num_samples в output_file.
    
    Используется хеширование для оптимизации сравнения.
    """
    # Загрузка данных
    print("Загрузка файлов...")
    source_df = pd.read_csv(source_file)
    comparison_df = pd.read_csv(comparison_file)
    
    # Получение столбца с текстами для сравнения
    comparison_texts = set()
    
    # Хеширование текстов из comparison_file для быстрого поиска
    print("Создание хеш-таблицы для быстрого сравнения...")
    for text in comparison_df.iloc[:, 0]:  # Первый столбец comparison_file
        # Используем хеш для ускорения сравнения
        text_hash = hashlib.md5(str(text).encode()).hexdigest()
        comparison_texts.add(text_hash)
    
    # Поиск уникальных текстов
    print("Поиск уникальных текстов...")
    unique_texts = []
    
    for text in source_df['text']:  # Второй столбец source_file
        text_hash = hashlib.md5(str(text).encode()).hexdigest()
        
        if text_hash not in comparison_texts:
            unique_texts.append(text)
            
            if len(unique_texts) >= num_samples:
                break
    
    # Сохранение результатов
    print(f"Найдено {len(unique_texts)} уникальных текстов")
    result_df = pd.DataFrame({'unique_text': unique_texts})
    result_df.to_csv(output_file, index=False)
    print(f"Результаты сохранены в {output_file}")
    
    return len(unique_texts)

In [12]:
# Использование функции
if __name__ == "__main__":
    source_file = "improvement_candidates_5.csv"
    comparison_file = "finetuning_text_pairs_3.csv"
    output_file = "unique_texts_500.csv"
    
    count = find_unique_texts(source_file, comparison_file, output_file, 300)
    print(f"Готово! Сохранено {count} уникальных текстов.")

Загрузка файлов...
Создание хеш-таблицы для быстрого сравнения...
Поиск уникальных текстов...
Найдено 300 уникальных текстов
Результаты сохранены в unique_texts_500.csv
Готово! Сохранено 300 уникальных текстов.


In [22]:
import pandas as pd
import openpyxl
from openpyxl.styles import Alignment

def format_csv_to_excel(csv_file, excel_file):
    """
    Преобразует готовый CSV файл в Excel с форматированием для русского текста.
    Тексты размещаются в ячейке с автоматическим переносом строк.
    
    Args:
        csv_file (str): Путь к исходному CSV файлу
        excel_file (str): Путь для сохранения Excel файла
    """
    # Загружаем CSV файл с правильной кодировкой для русского языка
    df = pd.read_csv(csv_file, encoding='utf-8')
    
    # Сначала сохраняем в Excel формат
    df.to_excel(excel_file, index=False, engine='openpyxl')
    
    # Открываем файл для форматирования
    wb = openpyxl.load_workbook(excel_file)
    ws = wb.active
    
    # Применяем форматирование к каждой ячейке
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = Alignment(
                wrap_text=True,        # Включаем перенос текста
                vertical='top',        # Выравнивание по верху
                horizontal='left'      # Выравнивание по левому краю
            )
    
    # Устанавливаем оптимальную ширину для столбцов с русским текстом
    for column in ws.columns:
        column_letter = column[0].column_letter
        ws.column_dimensions[column_letter].width = 100  # Широкие столбцы для удобства чтения
    
    # Сохраняем отформатированный файл
    wb.save(excel_file)
    print(f"Файл успешно преобразован и сохранен: {excel_file}")

# Использование функции
if __name__ == "__main__":
    csv_file = "unique_texts_500.csv"
    excel_file = "unique_texts_500_formatted.xlsx"
    
    format_csv_to_excel(csv_file, excel_file)

Файл успешно преобразован и сохранен: unique_texts_500_formatted.xlsx
